In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from scipy import stats
from scipy.stats import norm, skew

In [3]:
import seaborn as sb

In [4]:
#EDA
#possible Missing values for columns
miss_val=['NaN']

In [5]:
#Loading Data

In [6]:
aisles=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/aisles.csv', na_values=miss_val)
departments=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/departments.csv', na_values=miss_val)
products=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/products.csv', na_values=miss_val)
order_products_prior=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/order_products__prior.csv', na_values=miss_val)
order_products_train=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/order_products__train.csv', na_values=miss_val)
orders=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/orders.csv', na_values=miss_val)

print('train:', order_products_prior.shape, 'test:', order_products_train.shape)

train: (32434489, 4) test: (1384617, 4)


In [46]:
aisles.count

<bound method DataFrame.count of      aisle_id                       aisle
0           1       prepared soups salads
1           2           specialty cheeses
2           3         energy granola bars
3           4               instant foods
4           5  marinades meat preparation
..        ...                         ...
129       130    hot cereal pancake mixes
130       131                   dry pasta
131       132                      beauty
132       133  muscles joints pain relief
133       134  specialty wines champagnes

[134 rows x 2 columns]>

In [47]:
departments.isnull().values.any()
departments.head(15)

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [48]:
products.isnull().values.any()
products.head(10)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
5,6,Dry Nose Oil,11,11
6,7,Pure Coconut Water With Orange,98,7
7,8,Cut Russet Potatoes Steam N' Mash,116,1
8,9,Light Strawberry Blueberry Yogurt,120,16
9,10,Sparkling Orange Juice & Prickly Pear Beverage,115,7


In [49]:
order_products_prior.isnull().values.any()
order_products_prior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
order_id             int64
product_id           int64
add_to_cart_order    int64
reordered            int64
dtypes: int64(4)
memory usage: 989.8 MB


In [50]:
order_products_train.isnull().values.any()
order_products_train.head(5)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [12]:
# Data Preprocessing

In [13]:
orders.fillna(-1, inplace=True) # Fill missing values with -1

In [14]:
# Combining dataframes orders_prior and orders_train
order_products_total=pd.concat([order_products_prior, order_products_train], axis=0)
order_products_total.shape

(33819106, 4)

In [15]:
order_products_usr=pd.merge(orders, order_products_total, on='order_id')
order_products_usr.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,-1.0,196,1,0
1,2539329,1,prior,1,2,8,-1.0,14084,2,0
2,2539329,1,prior,1,2,8,-1.0,12427,3,0
3,2539329,1,prior,1,2,8,-1.0,26088,4,0
4,2539329,1,prior,1,2,8,-1.0,26405,5,0
5,2398795,1,prior,2,3,7,15.0,196,1,1
6,2398795,1,prior,2,3,7,15.0,10258,2,0
7,2398795,1,prior,2,3,7,15.0,12427,3,1
8,2398795,1,prior,2,3,7,15.0,13176,4,0
9,2398795,1,prior,2,3,7,15.0,26088,5,1


In [ ]:
# Data Visualization

In [51]:
#Product Popularity by ALL users
most_freq_prod=order_products_usr.groupby('product_id').size().sort_values(ascending=False).reset_index()
print(most_freq_prod)

       product_id       0
0           24852  491291
1           13176  394930
2           21137  275577
3           21903  251705
4           47209  220877
...           ...     ...
49680        7626       1
49681       14485       1
49682       43098       1
49683       11254       1
49684       36233       1

[49685 rows x 2 columns]


In [17]:
# Total count of each product for every user
freq_user_product=order_products_usr.groupby('user_id')['product_id'].value_counts()
    #freq_user_product[i]=order_products_usr[order_products_usr['user_id']==i]['product_id'].value_counts()
    #most_freq_product_user[i]=freq_user_product[i].index[0]
    #print("Most frequent product for User {} is {}\n\n".format(i,most_freq_product_user[i]))
print(freq_user_product)

user_id  product_id
1        196           11
         10258         10
         12427         10
         25133          9
         13032          4
                       ..
206209   41665          1
         42606          1
         44325          1
         48370          1
         48697          1
Name: product_id, Length: 13863746, dtype: int64


In [18]:
# Total number of unique orders for every user
total_orders_user=order_products_usr.groupby('user_id')['order_id'].nunique().reset_index().rename(columns={'order_id':'total_unique_orders'})
print(total_orders_user)

        user_id  total_unique_orders
0             1                   11
1             2                   15
2             3                   12
3             4                    5
4             5                    5
...         ...                  ...
206204   206205                    4
206205   206206                   67
206206   206207                   16
206207   206208                   49
206208   206209                   14

[206209 rows x 2 columns]


In [52]:
# Total number of orders for every user
total_orders_user2=order_products_usr.groupby('user_id')['order_id'].count().reset_index().rename(columns={'order_id':'count_orders'})
#print(total_orders_user2)

# Product details of every order for every user
total_orders_user3=order_products_usr.groupby('user_id')['order_id'].value_counts() 
print(total_orders_user3)

user_id  order_id
1        1187899     11
         2550362      9
         431534       8
         2295261      6
         2398795      6
                     ..
206209   272231       8
         1854736      8
         2307371      3
         2558525      3
         3186442      2
Name: order_id, Length: 3346083, dtype: int64


In [53]:
torders=0
for key, value in total_orders_user3.items():
    if (key[0]==1):
        torders+=value

print(torders)

70


In [22]:
# Unique Product Count for every user
unique_product_user=order_products_usr.groupby('user_id')['product_id'].nunique().reset_index().rename(columns={'product_id':'num_unique_products'})
print(unique_product_user)    

        user_id  num_unique_products
0             1                   19
1             2                  121
2             3                   33
3             4                   17
4             5                   28
...         ...                  ...
206204   206205                   37
206205   206206                  150
206206   206207                   92
206207   206208                  198
206208   206209                   72

[206209 rows x 2 columns]


In [54]:
#new_product_ratio for every user = unique products/total orders

nd=pd.merge(unique_product_user, total_orders_user2, on="user_id")
nd['new_product_ratio'] = nd.apply(lambda row: row.num_unique_products/row.count_orders*100, axis=1)
print(nd)

        user_id  num_unique_products  count_orders  new_product_ratio
0             1                   19            70          27.142857
1             2                  121           226          53.539823
2             3                   33            88          37.500000
3             4                   17            18          94.444444
4             5                   28            46          60.869565
...         ...                  ...           ...                ...
206204   206205                   37            51          72.549020
206205   206206                  150           285          52.631579
206206   206207                   92           223          41.255605
206207   206208                  198           677          29.246677
206208   206209                   72           137          52.554745

[206209 rows x 4 columns]


In [ ]:
#Product ordering ratio for every user = total number of orders containing product/ total number of orders
#How many times a user has reordered a product?

order_contains_product=order_products_usr.groupby(['user_id', 'product_id'])['order_id'].agg(['count']).reset_index().rename(columns={'count': "orders_with_product"})
total_user_order=order_products_usr.groupby('user_id')['order_id'].agg(['count']).reset_index().rename(columns={'count': "total_orders"})
nro=pd.merge(order_contains_product, total_user_order, on="user_id")
nro['product_order_ratio']=nro.apply(lambda row: row.orders_with_product/row.total_orders*100, axis=1)
print(nro)

In [ ]:
print(total_user_order.iloc[0]) #iloc takes specific row number as paramter.

In [27]:
# Frequency of products ordered by every user in each order
#user_order_products22=order_products_usr.groupby(['user_id', 'order_id'])['product_id'].count() # returns a Series
user_order_products=order_products_usr.groupby(['user_id', 'order_id'])['product_id'].agg(['count',  'mean']) # returns a Dataframe
for a in user_order_products.itertuples():
    print(a[0], a[1], a[2])
    break

(1, 431534) 8 16303.125


In [28]:
# Most Frequent time for every user
user_time=order_products_usr.groupby('user_id')['order_hour_of_day'].value_counts()

In [29]:
#Most frequent Time of purchase for the shop
most_popular_time=order_products_usr.groupby('order_hour_of_day')['user_id'].count()

In [30]:
# Most frequent day of purchase for every user
user_day=order_products_usr.groupby('user_id')['order_dow'].value_counts()

In [31]:
# Most frequent day of purchase for shop
most_popular_day=order_products_usr.groupby('order_dow')['user_id'].count()

In [32]:
order_products_usr.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,-1.0,196,1,0
1,2539329,1,prior,1,2,8,-1.0,14084,2,0
2,2539329,1,prior,1,2,8,-1.0,12427,3,0
3,2539329,1,prior,1,2,8,-1.0,26088,4,0
4,2539329,1,prior,1,2,8,-1.0,26405,5,0


In [33]:
departments.head(2)

,department_id,department
0,1,frozen
1,2,other


In [ ]:
#Most ordered product in every department
most_popular_product_department=

In [ ]:
#Did any user reorder their previous order

In [ ]:
#Most frequent time when users purchase

In [ ]:
#Times a customer orders from the same department. Wrt to customer

In [ ]:
#Days a customer orders the same products

In [ ]:
#Times a product is ordered from the same department. Wrt product